# Approxaimation of summator by LSTM and NN

In [25]:
import numpy as np
import torch
from torch import nn
from torch.nn import LSTM, Linear
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## Generate dataset

In [26]:
DATASET_SIZE = 1000
X = np.random.randint(1, 100, size=(DATASET_SIZE, 2))
y = X.sum(axis=1).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

## LSTM

### Model

In [27]:
class LSTMSumModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=2, hidden_size=6, num_layers=2, batch_first=True)
        self.fc = nn.Linear(6, 1)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len=1, input_size=2)
        lstm_out, _ = self.lstm(x)
        last_out = lstm_out[:, -1, :]  # Take last timestep
        return self.fc(last_out)

### Training

In [36]:
model = LSTMSumModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
    
    # Процесс обучения
def train_model():
    for epoch in range(2000):
        model.train()
        total_loss = 0
        
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss/len(train_loader):.4f}')
    
    return model

In [37]:
model = train_model()

Epoch 0, Loss: 11632.5821
Epoch 100, Loss: 8689.0570
Epoch 200, Loss: 6638.3674
Epoch 300, Loss: 4951.6690
Epoch 400, Loss: 3585.3942
Epoch 500, Loss: 2510.6459
Epoch 600, Loss: 1695.5921
Epoch 700, Loss: 1101.9607
Epoch 800, Loss: 691.6847
Epoch 900, Loss: 418.6979
Epoch 1000, Loss: 242.2986
Epoch 1100, Loss: 132.9607
Epoch 1200, Loss: 67.7157
Epoch 1300, Loss: 31.3220
Epoch 1400, Loss: 13.4353
Epoch 1500, Loss: 5.9236
Epoch 1600, Loss: 2.3846
Epoch 1700, Loss: 1.0997
Epoch 1800, Loss: 0.5966
Epoch 1900, Loss: 0.7934


In [38]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.1414


## Dense NN

In [48]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [49]:
class DenseSumModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = Linear(in_features=2, out_features=1)
        
    def forward(self, x):
        return self.fc(x)

In [51]:
model = DenseSumModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

for epoch in range(100):
    model.train()
    total_loss = 0
    
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {total_loss/len(train_loader):.4f}')        

Epoch 0, Loss: 6823.5762
Epoch 10, Loss: 6.3928
Epoch 20, Loss: 0.2335
Epoch 30, Loss: 0.0086
Epoch 40, Loss: 0.0063
Epoch 50, Loss: 0.0060
Epoch 60, Loss: 0.0055
Epoch 70, Loss: 0.0051
Epoch 80, Loss: 0.0046
Epoch 90, Loss: 0.0042


In [52]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.0033
